<a href="https://colab.research.google.com/github/mofnandos/CS5224/blob/main/CS5224_Project_DataDotGov_APIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CS224 Goverment API

In [ ]:
try:
  ! pip install pgeocode
  ! pip install haversine
  ! pip install geopy
  ! pip install pyproj
except:
  print("Something not installed")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pgeocode
import requests # HTTP requests (GET / POST)
import datetime
import pandas as pd
import numpy as np
import folium
import json
from pyproj import Transformer
from geopy.geocoders import Nominatim
from haversine import haversine, Unit
from IPython.display import display, Image

pd.set_option("display.max_colwidth", None)

## Helper Functions

In [ ]:
def address_to_coord(address):
  # calling the Nominatim tool
  loc = Nominatim(user_agent="GetLoc")
  
  # entering the location name
  getLoc = loc.geocode(address)
  
  # printing address
  # print(getLoc.address)
  # printing latitude and longitude
  place_lat = getLoc.latitude
  place_long = getLoc.longitude
  place_coord = (place_lat, place_long)
  # print(place_coord)

  return place_coord

In [ ]:
def postalcode_to_coord(postal_code):
  nomi = pgeocode.Nominatim('sg')
  place = nomi.query_postal_code(postal_code)

  place_lat = place['latitude']
  place_long = place['longitude']
  place_coord = (place_lat, place_long)
  # print(place_coord)
  return place_coord

In [ ]:
### Test code ###

address = "Bedok South Avenue 1" #user input value - destination location should be HDB area
postal_code = "469334" #user input value - destination location must be HDB block (example is Bedok South Avenue 1)

print(address_to_coord(address))
print(postalcode_to_coord(postal_code))

(1.3148778, 103.9335649)
(1.3168, 103.9324)


# Carpark Availability API

## Initialization: Getting the list of HDB Carpark

In [ ]:
def carpark_init(file_name):
  df_hdb_cp = pd.read_csv(file_name)
  # df_hdb_cp.head()

  transformer = Transformer.from_crs("EPSG:3414", "EPSG:4326")
  df_hdb_cp['lat_log'] = None

  for i in range(df_hdb_cp.shape[0]):
    df_hdb_cp['lat_log'][i] = transformer.transform(df_hdb_cp['y_coord'][i], df_hdb_cp['x_coord'][i])
  
  return df_hdb_cp

In [ ]:
### Test code ###
file_name = 'hdb-carpark-information.csv'
df_hdb_cp = carpark_init(file_name)

<ipython-input-6-885a3ba631e3>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hdb_cp['lat_log'][i] = transformer.transform(df_hdb_cp['y_coord'][i], df_hdb_cp['x_coord'][i])


## Finding the nearest carpark

In [ ]:
def nearest_carpark(place_coord, walking_dist_to_cp):
  nearest_carpark_list = []
  for i in range(df_hdb_cp.shape[0]):
    if haversine(df_hdb_cp['lat_log'][i], place_coord, unit=Unit.METERS) < walking_dist_to_cp:
      # print(df_hdb_cp['car_park_no'][i])
      nearest_carpark_list.append(df_hdb_cp['car_park_no'][i])
  return nearest_carpark_list

In [ ]:
### Test Code ###
walking_dist_to_cp = 1000 # distance in meters
place_coord = address_to_coord("Bedok South Avenue 1")
nearest_carpark_list = nearest_carpark(place_coord, walking_dist_to_cp)
print(nearest_carpark_list)

['B6T', 'B68', 'B81', 'B87', 'B88', 'B92', 'B93']


## Finding the nearest HDB carpark with availability data

In [ ]:
def hdb_carpark_availability(nearest_carpark_list):
  # Obtaining carpark availability
  today = datetime.datetime.today()
  params = {"date_time": today.strftime("%Y-%m-%dT%H:%M:%S")} # YYYY-MM-DD 
  car = requests.get('https://api.data.gov.sg/v1/transport/carpark-availability', params=params).json()
  # print(len(car['items'][0]['carpark_data'])) # Amount of instances retrieved per API call

  df = pd.DataFrame.from_dict(car['items'][0]['carpark_data'])
  # df.head()

  # set of carpark which we have availability details
  carpark_number_set = set()
  for i in range(df.shape[0]):
    carpark_number_set.add(df['carpark_number'][i])
  # print(carpark_number_set)

  carpark = {}

  if len(nearest_carpark_list) > 0:
    for i in range(len(nearest_carpark_list)):
      carpark_number = nearest_carpark_list.pop()
      if carpark_number in carpark_number_set:
        lots_available = df[df['carpark_number'] == carpark_number].iloc[0,0][0]['lots_available']
        carpark[carpark_number] = lots_available
        # print("Car Park Number {}: {} lots avaialble".format(carpark_number, lots_available))
  else:
    carpark['availability'] = 'No HDB carparks nearby'
  
  return carpark

In [ ]:
### Test Code###
carpark_availability = hdb_carpark_availability(nearest_carpark_list)
for carpark_number, lots_available in carpark_availability.items():
  print("Car Park Number {}: {} lots avaialble".format(carpark_number, lots_available))

Car Park Number B92: 281 lots avaialble
Car Park Number B88: 234 lots avaialble
Car Park Number B81: 433 lots avaialble


# Taxi Availability API



In [ ]:
# postal_code = "238801" #user input value - starting location (example is Ion Orchard)
# nomi = pgeocode.Nominatim('sg')
# place = nomi.query_postal_code(postal_code)

# place_lat = place['latitude']
# place_long = place['longitude']
# place_coord = (place_lat, place_long)
# print(place_coord)

In [ ]:
def taxi_availability(place_coord, dist):
  # Taxi availability data
  today = datetime.datetime.today() 
  params = {"date": today.strftime("%Y-%m-%d")} # YYYY-MM-DD 
  taxi = requests.get('https://api.data.gov.sg/v1/transport/taxi-availability', params=params).json()
  num_taxi_all_singapore = taxi['features'][0]['properties']['taxi_count']

  taxi_list = taxi['features'][0]['geometry']['coordinates']
  taxi_near = []

  for taxi in taxi_list:
    taxi_tup = (taxi[1], taxi[0])
    if haversine(taxi_tup, place_coord, unit=Unit.METERS) < dist:
      # print(taxi_tup)
      taxi_near.append(taxi)

  num_taxi_near_me = len(taxi_near)

  return num_taxi_all_singapore, num_taxi_near_me, taxi_near # (long,lat)

In [ ]:
### Test Code ###
dist = 500
place_coord = postalcode_to_coord("238801")
num_taxi_all_singapore, num_taxi_near_me, taxi_near = taxi_availability(place_coord, dist)
print("There are currently {} taxi available in Singapore".format(num_taxi_all_singapore))
print("There are current {} taxi near me (within {}m)".format(num_taxi_near_me, dist))
print(taxi_near)

There are currently 1461 taxi available in Singapore
There are current 4 taxi near me (within 500m)
[[103.83109, 1.30376], [103.831903966667, 1.3052605], [103.832953333333, 1.30446033333333], [103.83413, 1.30737]]


In [ ]:
def plot_taxi_near_me(taxi_near):
# Plotting taxi near my location
  sg_map = folium.Map([1.3521, 103.8198], zoom_start = 12, tiles="Stamen Terrain")
  for coord in taxi_near:
      folium.Marker([coord[1], coord[0]]).add_to(sg_map)
  sg_map.save('taxi_availability.html')

In [ ]:
### Test code ###
plot_taxi_near_me(taxi_near)

# Weather Forecast


In [ ]:
start_place_coord = postalcode_to_coord("638365")
dest_place_coord = postalcode_to_coord("469334")

In [ ]:
def get_weather_forecast():
  # 2-hour weather forecast data
  today = datetime.datetime.today() 
  params = {"date": today.strftime("%Y-%m-%d")} # YYYY-MM-DD 
  wx_forecast = requests.get('https://api.data.gov.sg/v1/environment/2-hour-weather-forecast', params=params).json()

  df = pd.DataFrame(wx_forecast['area_metadata']).merge(pd.DataFrame(wx_forecast['items'][-1]['forecasts']).reset_index(), how='inner', left_on='name', right_on='area')
  df = df[['area','forecast','label_location']]
  df['lat_log'] = None

  for i in range(df.shape[0]):
    df['lat_log'][i] = (df['label_location'][i]['latitude'], df['label_location'][i]['longitude'])

  df.drop(['label_location'], axis = 1, inplace = True)
  # df.head()
  return df


In [ ]:
### Test code ###
df_weather_forecast = get_weather_forecast()

In [ ]:
def trip_weather_forecast(df_weather_forecast, start_place_coord, dest_place_coord):
  nearest_to_start = None
  nearest_to_dest = None

  dist_to_start = 1e10
  dist_to_dest = 1e10

  for i in range(df_weather_forecast.shape[0]):
    i_to_start = haversine(df_weather_forecast['lat_log'][i], start_place_coord, unit=Unit.METERS)
    if i_to_start < dist_to_start:
      dist_to_start = i_to_start
      nearest_to_start = i
    i_to_dest = haversine(df_weather_forecast['lat_log'][i], dest_place_coord, unit=Unit.METERS) 
    if i_to_dest < dist_to_dest:
      dist_to_dest = i_to_dest
      nearest_to_dest = i

  return (df_weather_forecast['area'][nearest_to_start], df_weather_forecast['forecast'][nearest_to_start]), \
         (df_weather_forecast['area'][nearest_to_dest], df_weather_forecast['forecast'][nearest_to_dest])

In [ ]:
### Test code ###
start_weather_forecast, dest_weather_forecast = trip_weather_forecast(df_weather_forecast, start_place_coord, dest_place_coord)
print(start_weather_forecast)
print(dest_weather_forecast)

('Jalan Bahar', 'Fair (Night)')
('Bedok', 'Fair (Night)')
